## Python and GBQ

This exercise gets us started using Python and GBQ together. In this notebook you'll practice the following common tasks: 

1. Connecting Python to GBQ
1. Listing tables in a data set
1. Uploading data 
1. Writing a query and returning the results
1. Deleting a table

This notebook assumes you have a single Dram Shop items detail report from Square. This file should be uncompressed and in the same folder as this code. 



In [ ]:
import os

import pandas as pd
import numpy as np
import pandas_gbq
import janitor

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

### GBQ Set Up

In this next section we connect to our GBQ project and list the data sets inside to test the connection.

In [ ]:
# These first two values will be different on your machine. 
service_path = "/Users/chandler/Dropbox/Teaching/"
service_file = 'umt-msba-037daf11ee16.json' # change this to your authentication information  

gbq_proj_id = 'umt-msba'  

# And this should stay the same. 
private_key = service_path + service_file

In [ ]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [ ]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [ ]:
for item in client.list_datasets() : 
    print(item.full_dataset_id)

### Listing Tables in a Dataset

In this section we'll list all the tables in one of the data sets. Verify this works by looking at the tables in the console.

In [ ]:
dataset_id = 'dram_shop'

tables = client.list_tables(dataset_id)  

for table in tables:
    print(table.table_id)
    
    
    

### Uploading Data

For this section, we'll use *your* project and dataset and upload the file that has our cleaned item names in it.

In [ ]:
gbq_proj_id = 'your_project'
dataset_id = 'your_dataset'

item_lu = pd.read_csv('item_lookup.txt',sep="\t")


In [ ]:
item_lu.head()

In [ ]:
item_lu.loc[item_lu['other_info'].notnull()]

In [ ]:
table_name = 'your_name'
table_id = ".".join([gbq_proj_id,dataset_id,table_name])
pandas_gbq.to_gbq(item_lu, table_id, project_id=gbq_proj_id,if_exists="replace") # let's discuss this last bit


### Querying from a table

Now let's see how we can connect and query against a table.

In [ ]:
# Let's query to see how many owners and records we now have loaded in

query = """SELECT *
           FROM `your_project.your_dataset.your_table`
           WHERE brewery = 'stone'"""

# And we execute queries with `client.query`
query_job = client.query(
    query,
    location="US",
)

for row in query_job :
    print("\t".join([str(item) for item in row]))
    


Or, alternatively, with this elegant line of pandas code.

In [ ]:
stone_lu = client.query(query).to_dataframe()

In [ ]:
stone_lu

### Deleting a table

Be careful to do this on your own project! Deleting a table is relatively easy, though.

In [ ]:
table_id = ".".join([gbq_proj_id,dataset_id,table_name])

print(table_id)

In [ ]:
client.delete_table(table_id, not_found_ok=True)
print(f"Deleted {table_id}.")


Remember, you may need to refresh your GBQ console to see the changes you've wrought.